### ML Engingeering

In [ ]:
import hydra

In [ ]:
@hydra.main(
    config_path="./",
    config_name="config.yaml"
)
def hello(cfg):
    return cfg

In [ ]:
from omegaconf import OmegaConf

In [ ]:
OmegaConf.load("./config.yaml")

In [ ]:
from pydantic import root_validator, BaseModel

In [ ]:
class User(BaseModel):
    password1: str
    password2: str
    
    @root_validator(pre=True)
    def check_password(cls, params):
        pd1 = params.get("password1")
        pd2 = params.get("password2")
        
        if pd1 == pd2:
            return params
        else:
            raise

In [ ]:
from pymongo import MongoClient

In [ ]:
client = MongoClient(URL)

In [ ]:
db.get_collection(COLLECTION_NAME)

In [ ]:
step 1: add to dvc
step 2: git ignore
step 3: push dvc
step 4: add metadata to git
step 5: git push

In [ ]:
import hydra

In [ ]:
@hydra.main(
    config_path="./",
    config_name="config.yaml"
)
def hello(cfg):
    return cfg

/var/folders/pf/g3nr86yn4j71vzzmv6knwdhr0000gp/T/ipykernel_1398/584366940.py:1: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(


### Engineering

In [ ]:
step 1: create value on cpu
step 2: memory reservation on gpu
step 3: move to gpu
step 4: determine
step 5: launch kernel
step 6: do
step 7: copy results back to cpu
step 8: memory dellocation

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
class f(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return input.clone()
    
    @staticmethod
    def backward(ctx, grad_output):
        torch.distributed.all_reduce(grad_output)
        return grad_output

In [ ]:
class g(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        world_size = torch.distributed.get_world_size()
        inputs = [torch.zeros_like(input) for _ in range(world_size)]
        torch.distributed.all_gather(inputs, dim=-1)
        inputs = torch.cat(inputs, dim=-1)
        return inputs
    
    @staticmethod
    def backward(ctx, grad_output):
        world_size = torch.distributed.get_world_size()
        rank = torch.distributed.get_rank()
        
        dim_size = input.shape[-1]
        dim_size_per_partrition = dim_size // world_size
        
        grad_chunks = torch.split(grad_output, dim_size_per_partrition)
        return grad_chunks[rank]

In [ ]:
class ColumnParallelLinear(nn.Module):
    def __init__(self, input_size, output_size, world_size):
        super().__init__()
        self.input_size = input_size
        self.output_size_per_partrition = output_size // world_size
        
        self.weight = nn.Parameter(torch.empty(
            self.output_size_per_partrition,
            self.input_size
        ))
        self.bias = nn.Parameter(torch.empty(
            self.output_size_per_partrition
        ))
    
    def forward(self, input):
        input_parallel = f.apply(input)
        output_parallel = F.linear(input_parallel, self.weight, self.bias)
        outputs = g.apply(output_parallel)
        return outputs

In [ ]:
import time

In [ ]:
def profile_times(model, batch):
    records = [[] for _ in model]
    
    for idx, layer in enumerate(model):
        start_time = time.time()
        outputs = [layer(x) for x in batch]
        outputs_with_grad = [x for x in outputs if x.requires_grad]
        
        if outputs_with_grad:
            torch.autograd.backward(outputs_with_grad, outputs_with_grad)
        
        end_time = time.time()
        records[idx].append(end_time-start_time)
    
    return records

In [ ]:
step 1: partrition
step 2: move to rank
step 3: move to device
step 4: local optimizer
step 5: step
step 6: broadcast
step 7: sync 

In [ ]:
global memory > shared memory > register memory

In [ ]:
grid > block > threads

### AI

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
from torch.optim import Optimizer

In [ ]:
class CustomOptimizer(Optimizer):
    def __init__(self, params):
        defaults = dict()
        super().__init__(params, defaults)
    
    def step(self):
        for param_group in self.param_groups:
            for params in param_group["params"]:
                if params.requires_grad:
                    params.data = params.data * 6.9

In [ ]:
import jax

In [ ]:
jax.grad

In [ ]:
import numpy as np
from gym import Env
from gym.spaces import Discrete, Box

In [ ]:
class ShowerEnv(Env):
    def __init__(self):
        super().__init__()
        self.action_space = Discrete(3)
        self.observation_space = Box(
            low=np.array([0]),
            high=np.array([100])
        )
    
    def step(self, action):
        self.temperature -= action
        self.shower_length -= 1
        
        reward = 1 if 37 <= self.temperature <= 39 else 0
        done = True if self.shower_length == 0 else False
        info = {}
        return self.temperature, reward, done, info
    
    def reset(self):
        self.temperature = 20
        self.shower_length = 60

In [ ]:
from einops import einsum

In [ ]:
interference = einsum(
    W, W,
    "b f1 d, b f2 d -> b f1 f2"
)

In [ ]:
def generate_transitions(model, env):
    transitions = []
    
    state, _ = env.reset()
    done = False
    
    while done:
        action = model(state)
        new_state, reward, done, truncated, info = env.step(action)
        transitions.append(state, action, reward, done)
        
        new_state = state